In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv")
test=pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv")
sample=pd.read_csv("../input/tabular-playground-series-feb-2022/sample_submission.csv")

In [ ]:
sample.head()

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
%%capture
pip install autogluon

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
train_data = TabularDataset("../input/tabular-playground-series-feb-2022/train.csv")

train_data.head()

In [ ]:
label = 'target'
print("Summary of target variable: \n", train_data[label].describe())

In [ ]:
metric = 'accuracy'
import autogluon.core as ag

nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 20,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'layers': ag.space.Categorical([100], [1000], [200, 100], [300, 200, 100]),  # each choice for categorical hyperparameter 'layers' corresponds to list of sizes for each NN layer to use
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 1000,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 3600*9  # train various models for ~2 min
num_trials = 50  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using Bayesian optimization routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    train, holdout_frac=0.1, time_limit=time_limit,
    hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

In [ ]:
# save_path = './agModels-predictTarget'
# time_limit = 3600*9  # for quick demonstration only, you should set this to longest time you are willing to wait (in seconds)
# metric = 'accuracy'  # specify your evaluation metric here
# predictor = TabularPredictor(label,path=save_path, eval_metric=metric).fit(train_data, time_limit=time_limit, presets='best_quality')


In [ ]:
predictor.fit_summary(show_plot=True)

In [ ]:
test_data = TabularDataset("../input/tabular-playground-series-feb-2022/test.csv")
#y_test = test_data[label]  # values to predict
test_data_nolab = test_data.copy()  # delete label column to prove we're not cheating
test_data_nolab.head()

In [ ]:
#predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)

In [ ]:
results = predictor.fit_summary(show_plot=True)

In [ ]:
predictor.predict(test_data_nolab)

In [ ]:
pred_probs = predictor.predict(test_data_nolab)
pred_probs.head(5)

In [ ]:
pred_probs

In [ ]:
data=pd.concat([test['row_id'],pred_probs],axis=1, ignore_index=True)
data

In [ ]:
data.columns=['row_id','target']

In [ ]:
data

In [ ]:
data.set_index('row_id').to_csv("tps_feb_2022.csv")